#  The Shared Library with GCC

When your program is linked against a shared library, only a small table is created in the executable. Before the executable starts running, **the operating system loads the machine code needed for the external functions** - a process known as **dynamic linking.** 

    
* Dynamic linking makes executable files smaller and saves disk space, because `one` copy of a **library** can be **shared** between `multiple` programs. 


* Furthermore, most operating systems allows one copy of a shared library in memory to be used by all running programs, thus, saving memory. 


* The shared library codes can be upgraded without the need to recompile your program.


A **shared library** has file extension of 

   * **`.so`** (shared objects) in `Linux(Unixes)`
   
   
   * **`.dll** (dynamic link library) in `Windows`. 


## 1: Building the shared library

The shared library we will build consist of a single source file: `SumArray.c/h`

We will compile the C file with `Position Independent Code( PIC )` into a shared library。

GCC assumes that all libraries 
   
* `start` with `lib`

* `end`  with `.dll`(windows) or `.so`(Linux)，

so, we should name the  shared library begin with `lib prefix` and the `.so/.dll` extensions.

* libSumArray.dll(Windows)

* libSumArray.so(Linux)


In [1]:
%%file ./demo/src/SumArray.h

#ifndef SUMARRAY_H
#define SUMARRAY_H

int sum(int array[], int size);

#endif

Overwriting ./demo/src/SumArray.h


In [2]:
%%file ./demo/src/SumArray.c

#include "SumArray.h"

/* Function definition
   Return the sum of the given array
*/

int sum(int array[], int size)
{
    int sum = 0;
    int i;
    for (i = 0; i < size; ++i) {
        sum += array[i];
    }
    return sum;
}

Overwriting ./demo/src/SumArray.c


### 1.1 Building DLL for  Windows

In [3]:
!gcc -c -O3 -Wall -fPIC -o ./demo/bin/SumArray.o  ./demo/src/SumArray.c
!gcc -shared -o ./demo/bin/libSumArray.dll  ./demo/bin/SumArray.o

**Makefile of Building DLL for Windows**

In [5]:
%%file ./demo/makefile-SumArray-dll

CC=gcc
CFLAGS=-O3 -Wall -fPIC 

SRCDIR= ./demo/src/
OBJDIR= ./demo/obj/
BINDIR= ./demo/bin/

all: libdll

libdll: obj
	 $(CC) -shared -o $(BINDIR)libSumArray.dll $(OBJDIR)SumArray.o
	 del .\demo\obj\SumArray.o
    
obj: ./demo/src/SumArray.c
	 $(CC) -c $(CFLAGS)  -o $(OBJDIR)SumArray.o $(SRCDIR)SumArray.c
     
clean:
	 del .\demo\src\libSumArray.dll

Writing ./demo/makefile-SumArray-dll


In [6]:
!make -f ./demo/makefile-SumArray-dll

gcc -c -O3 -Wall -fPIC   -o ./demo/obj/SumArray.o ./demo/src/SumArray.c
gcc -shared -o ./demo/bin/libSumArray.dll ./demo/obj/SumArray.o
del .\demo\obj\SumArray.o


In [7]:
!dir .\demo\bin\libSum*.dll

 驱动器 F 中的卷是 cmh
 卷的序列号是 9C25-3306

 F:\SEU\SEE\PySEE\home\notebook\demo\bin 的目录

2019/04/28  21:23            47,790 libSumArray.dll
               1 个文件         47,790 字节
               0 个目录 103,468,867,584 可用字节


### 1.2 Building SO for  Linux

In [ ]:
!gcc -c -O3 -Wall -fPIC -o ./demo/obj/SumArray.o ./demo/gcc/SumArray.c
!gcc -shared -o ./cdemo/bin/libSumArray  ./demo/obj/SumArray.o 

In [ ]:
!ls ./demo/bin/libSumArray.*


* `-c`: compile into object file with default name : funs.o.

      By default, the object file has the same name as the source file with extension of ".o" 
  
  
* `-O3`: Optimize yet more.

      turns on all optimizations specified by -O2 and also turns on the -finline-functions, -fweb, -frename-registers and -funswitch-loops optionsturns  on  all  optimizations   
     
  
* `-Wall`: prints "all"  compiler's warning message. 

      This option should always be used, in order to generate better code.


* **`-fPIC`** : stands for `Position Independent Code`(位置无关代码)
   
   the generated machine code is `not dependent` on being located at a `specific address` in order to `work`.
   
   Position-independent code can be `executed` at `any memory address`
    
     
* **-shared:** creating a shared library


**Makefile of Building SO for Linux**

In [ ]:
%%file ./code/makefile-SumArray-so


CC=gcc
CFLAGS=-O3 -Wall -fPIC

SRCDIR= ./demo/src/
OBJDIR= ./demo/obj/
BINDIR= ./demo/bin/

all: libdll

libdll: obj
	 $(CC) -shared -o $(BINDIR)libSumArray.dll $(OBJDIR)SumArray.o
	 rm -f ./demo/obj/SumArray.o
    
obj: ./demo/src/SumArray.c
	 $(CC) -c $(CFLAGS)  -o $(OBJDIR)SumArray.o $(SRCDIR)SumArray.c
     
clean:
	 rm -f ./demo/src/libSumArray.dll


In [ ]:
!make -f ./code/makefile-SumArray-so

In [ ]:
!ls ./code/bin/libSum*.so

## 2 Building a Client Executable 

### 2.1 Header Files and Libraries 

* `Header File`: When compiling the program, the **compiler** needs the **header** files to compile the source codes;

* `libraries`: the **linker** needs the **libraries** to resolve external references from other object files or libraries. 

The `compiler` and `linker` will not find the `headers/libraries` unless you set **the appropriate options**

* **1 Searching for Header Files**

   **`-Idir`:** The include-paths are specified via **-Idir** option (`uppercase` 'I' followed by the directory path or environment variable **CPATH**). 
   
   
* **2 Searching for libraries Files**

   **`-Ldir`**: The library-path is specified via **-Ldir** option (`uppercase` 'L' followed by the directory path(or environment variable **LIBRARY_PATH**). 


* **3 Linking the library**

   **`-llibname`**: Link with the library name **without** the `lib` prefix and the `.so/.dll` extensions.
  
        Windows
    ```bash
         -I./demo/src/ -L./demo/bin/ -lSumArray
    ```
        Linux
    ```bash
         -I./demo/src/ -L./demo/bin/ -lSumArray -Wl,-rpath=./demo/bin/  
    ```

   * **`-Wl,option`**

    Pass option as an option to the **linker**. If option contains `commas`, it is split into multiple options at the commas. You can use this syntax to pass an argument to the option. For example, -Wl,-Map,output.map passes -Map output.map to the linker. When using the GNU linker, you can also get the same effect with `-Wl,-Map=output.map'.
    
   * **`-rpath=dir`** 

    **Add a directory to the runtime library search path**. This is used when linking an ELF executable with shared objects. All -rpath arguments are concatenated and passed to the runtime linker, which uses them to locate shared objects at runtime. The -rpath option is also used when locating shared objects which are needed by shared objects explicitly included in the link;
   


### 2.2 Calling the Shared Library

The following source code `"mainSum.c"` demonstrates calling the DLL's functions: 

**NOTE:** `mainSum.c` is the **same** code in multi-source example

In [ ]:
%%file ./demo/src/mainSum.c

#include <stdio.h> 
#include "SumArray.h"

int main() {
    
     int a1[] = {8, 4, 5, 3, 2};
     printf("sum is %d\n", sum(a1, 5));  // sum is 22
     return 0;
}

#### 2.2.1 Windows

In [ ]:
!gcc -c -o ./demo/obj/mainSum.o ./demo/src/mainSum.c 
!gcc -o  ./demo/bin/mainSum ./demo/obj/mainSum.o -I./demo/src/ -L./demo/bin/ -lSumArray

**Makefile under Windows**

In [ ]:
%%file ./demo/makefile-call-dll

SRCDIR= ./demo/src/
OBJDIR= ./demo/obj/
BINDIR= ./demo/bin/

all: mainexe

clean:
	del .\demo\bin\mainSum.exe

mainexe: sumobj $(SRCDIR)SumArray.h 
	gcc -o $(BINDIR)mainSum.exe $(OBJDIR)mainSum.o -I$(SRCDIR) -L$(BINDIR) -lSumArray
	del .\demo\obj\mainSum.o

sumobj: $(SRCDIR)mainSum.c 
	gcc -c -o $(OBJDIR)mainSum.o $(SRCDIR)mainSum.c 

In [ ]:
!make -f ./demo/makefile-call-dll

In [ ]:
!.\demo\bin\mainSum

#### 2.2.2 Linux

In [ ]:
!gcc -c -o ./demo/obj/mainSum.o ./demo/obj/mainSum.c 
!gcc -o  ./demo/bin/mainSum ./demo/obj/mainSum.o -I./demo/obj/ -L./demo/bin/ -lSumArray -Wl,-rpath=./demo/bin/

**makefile under Linux**

In [ ]:
%%file ./demo/makefile-call-so


SRCDIR= ./demo/src/
OBJDIR= ./demo/obj/
BINDIR= ./demo/bin/

all: main

clean:
	rm -f ./demo/bin/mainSum.exe

main: sumobj $(SRCDIR)SumArray.h 
	gcc -o $(BINDIR)mainSum.exe $(OBJDIR)mainSum.o -I$(SRCDIR) -L$(BINDIR) -lSumArray -Wl,-rpath=./code/bin/ 
	rm -f ./demo/obj/mainSum.o

sumobj: $(SRCDIR)mainSum.c 
	gcc -c -o $(OBJDIR)mainSum.o $(SRCDIR)mainSum.c    
     

In [ ]:
!make -f ./demo/makefile-call-so

In [ ]:
!./demo/bin/mainSum

## 3 Building  a Shared Library with Multi-source

### 3.1  The multi-source files
The shared library we will build consist of the multi-source files

* funs.c/h

* SumArray.c/h

In [ ]:
%%file ./demo/src/funs.h

#ifndef FUNS_H
#define FUNS_H

double dprod(double *x, int n);
int factorial(int n);

#endif

In [ ]:
%%file ./demo/src/funs.c

#include "funs.h"

// x[0]*x[1]*...*x[n-1]
double dprod(double *x, int n)
{
    double y = 1.0;
    for (int i = 0; i < n; i++)
    {
        y *= x[i];
    }
    return y;
}

// The factorial of a positive integer n, denoted by n!, is the product of all positive integers less than or equal to n. 
//  For example,5!=5*4*3*2*1=120
//  The value of 0! is 1 
int factorial(int n)
{
    if (n == 0 ) {
        return 1;
    }
    else 
    {
        return n * factorial(n - 1);
    }
}

### 3.2 Building  into One Shared Library

#### 3.2.1 Windows's DLL

In [ ]:
!gcc -c -O3 -Wall -fPIC -o ./demo/obj/funs.o ./demo/src/funs.c 
!gcc -c -O3 -Wall -fPIC -o ./demo/obj/SumArray.o  ./demo/src/SumArray.c
!gcc -shared -o ./demo/bin/libmultifuns.dll  ./demo/obj/funs.o  ./demo/obj/SumArray.o

In [ ]:
!dir .\demo\bin\libmulti*.dll

#### 3.2.2 Building with makefile


In [ ]:
%%file ./demo/makefile-libmultifun

CC=gcc
CFLAGS=-O3 -Wall -fPIC

SRCDIR= ./demo/src/
OBJDIR= ./demo/obj/
BINDIR= ./demo/bin/

all: libmultifuns.dll

libmultifuns.dll: multifunsobj
	 $(CC) -shared -o $(BINDIR)libmultifuns.dll $(OBJDIR)funs.o $(OBJDIR)SumArray.o
	 del .\demo\obj\funs.o .\demo\obj\SumArray.o
    
multifunsobj: $(SRCDIR)funs.c $(SRCDIR)SumArray.c
	$(CC) -c $(CFLAGS) -o $(OBJDIR)SumArray.o $(SRCDIR)SumArray.c
	$(CC) -c $(CFLAGS) -o $(OBJDIR)funs.o $(SRCDIR)funs.c 
     
     
clean:
	 del .\demo\bin\libmultifuns.dll

In [ ]:
!make -f ./demo/makefile-libmultifun

The result is a compiled shared library **`libmultifuns.dll`**

#### 3.2.3  makefile with more vars

In [ ]:
%%file ./code/makefile-libmultifun

CC=gcc
CFLAGS=-O3 -Wall -fPIC  

SRCDIR= ./demo/src/
OBJDIR= ./demo/obj/
BINDIR= ./demo/bin/

INC = -I$(SRCDIR) 

SRCS= $(SRCDIR)funs.c \
      $(SRCDIR)SumArray.c 

all: libmultifuns.dll

libmultifuns.dll: multifunsobj
	 $(CC)  -shared -o $(BINDIR)libmultifuns.dll funs.o SumArray.o
	 del funs.o SumArray.o
    
multifunsobj: 
	 $(CC) -c $(CFLAGS) $(INC) $(SRCS) 
     
clean:
	 del .\demo\bin\libmultifuns.dll

In [ ]:
!make -f ./code/makefile-libmultifun

### 3.3 Building a client executable 

The following source code `"mainMultifuns.c"` demonstrates calling the DLL's functions:


In [ ]:
%%file ./demo/src/mainMultifuns.c

#include <stdio.h> 
#include "SumArray.h"
#include "funs.h"

int main() {
    
     int a1[] = {8, 4, 5, 3, 2};
     printf("sum is %d\n", sum(a1, 5));  // sum is 22
    
     double a2[] = {8.0, 4.0, 5.0, 3.0, 2.0};
     printf("dprod is %f\n", dprod(a2, 5));  // dprod is 960
    
     int n =5;
     printf("the factorial of  %d is %d\n",n,factorial(n));  // 5!=120
     return 0;
}

In [ ]:
!gcc -c -o ./demo/obj/mainMultifuns.o ./demo/src/mainMultifuns.c 
!gcc -o  ./demo/bin/mainMultifuns ./demo/obj/mainMultifuns.o -I./demo/src/ -L./demo/bin/ -lmultifuns

In [ ]:
!.\demo\bin\mainMultifuns

## Reference

* GCC (GNU compilers) http://gcc.gnu.org

  * GCC Manual  http://gcc.gnu.org/onlinedocs

  * An Introduction to GCC  http://www.network-theory.co.uk/docs/gccintro/index.html.

  * GCC and Make：Compiling, Linking and Building C/C++ Applications http://www3.ntu.edu.sg/home/ehchua/programming/cpp/gcc_make.html

  * MinGW-W64 (GCC) Compiler Suite: http://www.mingw-w64.org/doku.php


* C/C++ for VS Code https://code.visualstudio.com/docs/languages/cpp

* C/C++ Preprocessor Directives http://www.cplusplus.com/doc/tutorial/preprocessor/


* What is a DLL and How Do I Create or Use One? http://www.mingw.org/wiki/DLL

